# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
# import libraries
import string
import pickle

import pandas as pd
import numpy as np

from sqlalchemy import create_engine

import nltk
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [24]:
nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table("Message", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df.message
Y = df.iloc[:, 5:]

X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [6]:
import re

skip_tokens = stopwords.words('english') + ["'s", "n't"] + list(string.punctuation)

def tokenize(text):
    lemm = WordNetLemmatizer()
    
    text = word_tokenize(text)
    text = [lemm.lemmatize(x).lower().strip() for x in text]
    text = filter(lambda x: x not in skip_tokens, text)
    
    return " ".join(text)

tokenize("Hello, My name wouldn't Heegyu Kim. He's name isn't perfectly perfects but that's ok and so on...")

'hello name would heegyu kim name perfectly perfect ok ...'

In [7]:
stopwords.words('english')
# string.punctuation

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=2)

In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [113]:
y_pred = pipeline.predict(X_test)

In [118]:
y_pred = pd.DataFrame(y_pred)
y_pred.columns = y_test.columns
y_pred.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
def test_prediction():
    y_pred = pipeline.predict(X_test)
    y_pred = pd.DataFrame(y_pred)
    y_pred.columns = y_test.columns
    
    reports = []
    for column in Y.columns:
        report = classification_report(y_test[column], y_pred[column])
        reports.append(report)

        print("Classification report for {}".format(column))
        print(report)

In [106]:
def score_multiclass(y, y_pred):
    columns = y.columns
    y_pred = np.array(y_pred)
    diff = np.array(y.values) - y_pred
    diff = pd.DataFrame(diff)
    diff.columns = columns
    
    for column in columns:
        diff[column] = diff[column].abs()
    
    class_error = diff.sum(axis=0) / len(diff)
    instance_error = diff.sum(axis=1) / len(columns)
    return class_error, instance_error

cls, ins = score_multiclass(y_test, y_pred)

In [107]:
cls

related                   0.245724
request                   0.160310
offer                     0.004276
aid_related               0.369394
medical_help              0.079404
medical_products          0.052011
search_and_rescue         0.028433
security                  0.018493
military                  0.030744
child_alone               0.000000
water                     0.061951
food                      0.105525
shelter                   0.091655
clothing                  0.016528
money                     0.022769
missing_people            0.012829
refugees                  0.036292
death                     0.044961
other_aid                 0.128525
infrastructure_related    0.066921
transport                 0.046232
buildings                 0.051896
electricity               0.022769
tools                     0.006357
hospitals                 0.011096
shops                     0.005663
aid_centers               0.012945
other_infrastructure      0.045307
weather_related     

In [108]:
ins

0       0.027778
1       0.027778
2       0.083333
3       0.055556
4       0.083333
5       0.111111
6       0.083333
7       0.111111
8       0.111111
9       0.027778
10      0.055556
11      0.166667
12      0.000000
13      0.027778
14      0.083333
15      0.027778
16      0.027778
17      0.111111
18      0.055556
19      0.083333
20      0.000000
21      0.166667
22      0.194444
23      0.083333
24      0.083333
25      0.027778
26      0.055556
27      0.027778
28      0.111111
29      0.027778
          ...   
8622    0.055556
8623    0.166667
8624    0.055556
8625    0.055556
8626    0.111111
8627    0.000000
8628    0.000000
8629    0.055556
8630    0.166667
8631    0.083333
8632    0.000000
8633    0.027778
8634    0.111111
8635    0.000000
8636    0.083333
8637    0.083333
8638    0.055556
8639    0.111111
8640    0.055556
8641    0.027778
8642    0.027778
8643    0.000000
8644    0.083333
8645    0.027778
8646    0.055556
8647    0.166667
8648    0.000000
8649    0.0000

In [110]:
accuracy_score(y_test, y_pred)

ValueError: multiclass-multioutput is not supported

### 6. Improve your model
Use grid search to find better parameters. 

In [36]:
parameters = {
#     'clf__estimator__max_depth': [10, 25, 50],
#     'clf__estimator__n_estimators': [10, 50, 100],

#     'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (5000, 10000, 50000)
}

cv = GridSearchCV(pipeline, parameters, verbose=1)

In [37]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed: 61.3min finished


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001CA5C8611E0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'vect__max_features': (5000, 10000, 50000)},
             verbose=1)

In [38]:
print('final params', cv.best_params_)   # best parameters
print('best score', cv.best_score_)      # best score

final params {'vect__max_features': 50000}
best score 0.371043040914741


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
test_prediction()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [112]:
with open("model.pkt", "wb") as f:
    pickle.dump(pipeline, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.